In [1]:
import pandas as pd

### Read in CSV.

In [2]:
# 1. Create a df from the vine table from Amazon_Vine_Analysis.
# vine_table.csv
vine_data_to_load = "vine_table.csv"
vine_df = pd.read_csv(vine_data_to_load)
vine_df

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,RTIS3L2M1F5SM,5,0,0,N,Y
1,R1ZV7R40OLHKD,5,0,0,N,Y
2,R3BH071QLH8QMC,1,0,1,N,Y
3,R127K9NTSXA2YH,3,0,0,N,Y
4,R32ZWUXDJPW27Q,4,0,0,N,Y
...,...,...,...,...,...,...
1785992,RPC430LWZJ60T,5,1,1,N,N
1785993,R347MZT5FH6HRJ,5,2,2,N,N
1785994,RJ5BETZP0VIUS,1,0,2,N,N
1785995,R85QTDO2KZMGO,1,1,3,N,N


<!-- Verify that there are Vine reviews in CSV. -->

In [3]:
# Groupby to verify that values contain "Y"
# vine_df.groupby(['vine'])['vine'].count()

## Apply universal filters, before splitting into Yes/No for Vine reviews.

### Total Votes >= 20

In [4]:
# Filter rows: total_votes >= 20
vine_df_gt20votes = vine_df[vine_df['total_votes'] >= 20]
# vine_df_gt20votes

In [5]:
# Groupby to verify that all values meet criteria.
vine_df_gt20votes.groupby(['total_votes'])['total_votes'].count().head()

total_votes
20    4226
21    3898
22    3544
23    3203
24    2933
Name: total_votes, dtype: int64

### % of Helpful Votes.

In [6]:
# Assign new calculated column: % of helpful_votes
vine_df_helpful = vine_df_gt20votes.assign(helpful_perc = vine_df_gt20votes['helpful_votes'] / vine_df_gt20votes['total_votes'])
# vine_df_helpful

In [7]:
# Filter rows: % of helpful_votes >= 50%
vine_df_helpful_gt50perc = vine_df_helpful[vine_df_helpful['helpful_perc'] > 0.5]

In [8]:
# Groupby to verify that all values meet criteria.
vine_df_helpful_gt50perc.groupby(['helpful_perc'])['helpful_perc'].count().head()

helpful_perc
0.504425    1
0.505263    1
0.505747    1
0.506098    1
0.506329    2
Name: helpful_perc, dtype: int64

In [9]:
vine_df_helpful_gt50perc.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,helpful_perc
55,R4PKAZRQJJX14,1,21,34,N,N,0.617647
74,R2CI0Y288CC7E2,1,21,35,N,Y,0.600000
209,R127WEQY2FM1T3,1,147,175,N,Y,0.840000
483,R2FJ94555FZH32,2,55,60,N,N,0.916667
537,R1U3AR67RE273L,1,51,65,N,Y,0.784615


## Split into Yes/No for Vine reviews.

## Vine = Yes

In [10]:
# Filter rows: vine == 'Y'
y_df = vine_df_helpful_gt50perc[vine_df['vine'] == "Y"]

C:\Users\LaStella\AppData\Local\Temp\ipykernel_12784\3829734516.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y_df = vine_df_helpful_gt50perc[vine_df['vine'] == "Y"]


In [11]:
# Groupby to verify that all values meet criteria.
y_df.groupby(['vine'])['vine'].count()

vine
Y    94
Name: vine, dtype: int64

### Vine = Yes:  Get sum of total votes.

In [12]:
# https://stackoverflow.com/questions/10373660/converting-a-pandas-groupby-output-from-series-to-dataframe
# df1.reset_index().groupby( [ "Name", "City"],as_index=False ).count()
y_df_tot = y_df.reset_index().groupby( [ "vine"],as_index=False ).count()

In [13]:
y_df_tot.head()

,vine,index,review_id,star_rating,helpful_votes,total_votes,verified_purchase,helpful_perc
0,Y,94,94,94,94,94,94,94


In [14]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.filter.html
# select columns by name
y_df_tot = y_df_tot.filter(items=['vine', 'total_votes'])

In [15]:
y_df_tot

,vine,total_votes
0,Y,94


### Vine = Yes: Get sum of 5-star votes.

In [16]:
# Filter rows: star_rating = 5
y_df_5star = y_df[y_df['star_rating'] == 5]

In [17]:
# Groupby to verify that all values meet criteria.
y_df_5star.groupby(['star_rating'])['star_rating'].count()

star_rating
5    48
Name: star_rating, dtype: int64

In [18]:
# https://stackoverflow.com/questions/10373660/converting-a-pandas-groupby-output-from-series-to-dataframe
# df1.reset_index().groupby( [ "Name", "City"],as_index=False ).count()
y_df_5star = y_df_5star.reset_index().groupby( [ "vine"],as_index=False ).count()

In [19]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.filter.html
# select columns by name
y_df_5star = y_df_5star.filter(items=['vine', 'helpful_perc'])

In [20]:
y_df_5star

,vine,helpful_perc
0,Y,48


### Vine = Yes: Merge dfs.

In [21]:
y_df_merge = pd.merge(y_df_tot, y_df_5star, on="vine")

In [22]:
y_df_merge

,vine,total_votes,helpful_perc
0,Y,94,48


In [23]:
# Assign new calculated column: % of 5-star reviews.
y_nums = y_df_merge.assign(percent_5star = (y_df_merge['helpful_perc'] / y_df_merge['total_votes']) * 100)

In [48]:
y_nums_fin = y_nums.rename(columns={"total_votes": "total_reviews", "helpful_perc": "5star_reviews"})

In [49]:
y_nums_fin

,vine,total_reviews,5star_reviews,percent_5star
0,Y,94,48,51.06383


## Vine = No

In [25]:
# Filter rows: vine == 'N'
n_df = vine_df_helpful_gt50perc[vine_df['vine'] == "N"]

C:\Users\LaStella\AppData\Local\Temp\ipykernel_12784\1436490038.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  n_df = vine_df_helpful_gt50perc[vine_df['vine'] == "N"]


In [26]:
# Groupby to verify that all values meet criteria.
n_df.groupby(['vine'])['vine'].count()

vine
N    39915
Name: vine, dtype: int64

### Vine = No: Get sum of total votes.

In [27]:
# https://stackoverflow.com/questions/10373660/converting-a-pandas-groupby-output-from-series-to-dataframe
# df1.reset_index().groupby( [ "Name", "City"],as_index=False ).count()
y_df_tot = n_df.reset_index().groupby( [ "vine"],as_index=False ).count()

In [28]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.filter.html
# select columns by name
y_df_tot = y_df_tot.filter(items=['vine', 'total_votes'])

In [29]:
y_df_tot

,vine,total_votes
0,N,39915


### Vine = No: Get sum of 5-star votes.

In [30]:
# Filter rows: star_rating = 5
n_df_5star = n_df[n_df['star_rating'] == 5]

In [31]:
# Groupby to verify that all values meet criteria.
n_df_5star.groupby(['star_rating'])['star_rating'].count()

star_rating
5    15556
Name: star_rating, dtype: int64

In [32]:
# https://stackoverflow.com/questions/10373660/converting-a-pandas-groupby-output-from-series-to-dataframe
# df1.reset_index().groupby( [ "Name", "City"],as_index=False ).count()
n_df_5star = n_df_5star.reset_index().groupby( [ "vine"],as_index=False ).count()

In [33]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.filter.html
# select columns by name
n_df_5star = n_df_5star.filter(items=['vine', 'helpful_perc'])

In [34]:
n_df_5star

,vine,helpful_perc
0,N,15556


### Vine = No: Merge dfs.

In [35]:
n_df_merge = pd.merge(y_df_tot, n_df_5star, on="vine")

In [36]:
n_df_merge

,vine,total_votes,helpful_perc
0,N,39915,15556


In [37]:
# Assign new calculated column: % of 5-star reviews.
n_nums = n_df_merge.assign(percent_5star = (n_df_merge['helpful_perc'] / n_df_merge['total_votes']) * 100)

In [45]:
n_nums_fin = n_nums.rename(columns={"total_votes": "total_reviews", "helpful_perc": "5star_reviews"})

In [51]:
n_nums_fin

,vine,total_reviews,5star_reviews,percent_5star
0,N,39915,15556,38.972817


## Display final totals and percentages.

In [50]:
print(y_nums_fin)
print(n_nums_fin)

  vine  total_reviews  5star_reviews  percent_5star
0    Y             94             48       51.06383
  vine  total_reviews  5star_reviews  percent_5star
0    N          39915          15556      38.972817
